In [1]:
import cirq
from mitiq import zne, benchmarks

# problem setup

In [2]:
# 1개의 qubit이 50개의 clifford군의 50개의 게이트를 통과하는 무작위 회로 생성 후 1번째 반환
circuit = benchmarks.generate_rb_circuits(n_qubits=1,num_cliffords=50)[0]
print(circuit)

0: ───X^0.5───Y^0───Y───X^-0.5───X^0.5───Y^0───Y───X^-0.5───Y^-0.5───X^0.5───Y^-0.5───X^-0.5───Y^0───Y^0.5───X^0───Y───X^0.5───Y^-0.5───X^0───Y^-0.5───X^-0.5───Y^0.5───X───Y^-0.5───X───Y^-0.5───X^-0.5───Y^-0.5───X^-0.5───Y^-0.5───X^0───Y^0.5───X^0───X^0.5───Y^0.5───X───Y^0.5───Y^-0.5───X^0.5───Y^-0.5───X^0.5───Y^0.5───X^-0.5───Y^0.5───Y^0.5───X^0.5───Y^0.5───X^0───X^0───Y^0.5───X^0───Y^-0.5───X^-0.5───Y^0.5───X^0.5───Y^-0.5───X^-0.5───Y───X───Y───X^-0.5───Y───X^-0.5───X^0───X^-0.5───Y^-0.5───Y───X───Y^-0.5───X^0───Y───X───X^0.5───Y^0───Y^-0.5───X^0───Y^-0.5───X^-0.5───X^0.5───Y^-0.5───X^0───X^0.5───Y^-0.5───Y───X^0───X^0.5───Y^0.5───X^-0.5───Y^0───Y^-0.5───X^0.5───X^0.5───Y^0───X^0.5───Y^0.5───Y^0.5───X^-0.5───X───Y^0───X───Y^0───


# make executer

In [3]:
def excute(circuit, noise_level = 0.005):
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    return(
        #A simulator for density matrices and noisy quantum circuits
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        #Returns the density matrix resulting from simulating the circuit
        .final_density_matrix[0,0]
        .real
    )

# apply ZNE and compute

In [9]:
true_value = excute(circuit,noise_level = 0.0)
noisy_value = excute(circuit)
zne_value = zne.execute_with_zne(circuit,excute)

print(f"Error without  Mitiq: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error with zne:    {abs((true_value - zne_value) / true_value):.3f}")


Error without  Mitiq: 0.246
Error with zne:    0.057


# select a noise scaling method

In [5]:
from mitiq.zne.scaling import fold_gates_at_random

folded = fold_gates_at_random(circuit,scale_factor = 2)
print("folded circuit : ", folded, sep='\n')

folded circuit : 
0: ───X^0.5───Y^0───Y───X^-0.5───X^0.5───X^-0.5───X^0.5───X^-0.5───X^0.5───Y^0───Y^0───Y^0───Y───X^-0.5───X^0.5───X^-0.5───Y^-0.5───Y^0.5───Y^-0.5───X^0.5───X^-0.5───X^0.5───Y^-0.5───X^-0.5───Y^0───Y^0───Y^0───Y^0.5───X^0───Y───Y───Y───X^0.5───Y^-0.5───X^0───Y^-0.5───X^-0.5───X^0.5───X^-0.5───Y^0.5───Y^-0.5───Y^0.5───X───Y^-0.5───Y^0.5───Y^-0.5───X───Y^-0.5───X^-0.5───Y^-0.5───Y^0.5───Y^-0.5───X^-0.5───X^0.5───X^-0.5───Y^-0.5───Y^0.5───Y^-0.5───X^0───X^0───X^0───Y^0.5───X^0───X^0.5───X^-0.5───X^0.5───Y^0.5───X───X───X───Y^0.5───Y^-0.5───Y^0.5───Y^-0.5───X^0.5───X^-0.5───X^0.5───Y^-0.5───Y^0.5───Y^-0.5───X^0.5───X^-0.5───X^0.5───Y^0.5───Y^-0.5───Y^0.5───X^-0.5───X^0.5───X^-0.5───Y^0.5───Y^-0.5───Y^0.5───Y^0.5───Y^-0.5───Y^0.5───X^0.5───X^-0.5───X^0.5───Y^0.5───X^0───X^0───Y^0.5───Y^-0.5───Y^0.5───X^0───Y^-0.5───X^-0.5───X^0.5───X^-0.5───Y^0.5───X^0.5───X^-0.5───X^0.5───Y^-0.5───X^-0.5───Y───Y───Y───X───X───X───Y───Y───Y───X^-0.5───X^0.5───X^-0.5───Y───X^-0.5───X^0.5───

# select a noise extrapolation method

In [8]:
from mitiq.zne.inference import LinearFactory

linear_fact = LinearFactory(scale_factors=[1.0,2.0])

mitigated_result = zne.execute_with_zne(circuit,excute,factory=linear_fact,scale_noise=fold_gates_at_random)
print(f"Error with mitigation (ZNE): {abs(true_value - mitigated_result):.{3}}")

Error with mitigation (ZNE): 0.12
